A chatbot can respond to queries in a chat like manner only if it has a memery. Otherwise each query will be trated as an independent input. Most LLM;s conversation system should be able to acess past conversations to make a good prediction(atleast conversations from past window).We can say the ability to store past information in conversation as Memory.

Langchain provide a lot of utilities for adding memory to the system.A memory need to support two things - reading and writing. A chain will interact with memory system twice in a given run:
   
   1. AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.

   2. AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.

<img src="images_dump/image_memory.png" width=700 height=300>


## Different types of memory


There are different ways we can build conversational memory. In context of langchain it is build on top of **ConversationChain**

Lets see a basic ConversationChain


   

In [1]:
from utils import *
OPENAI_API_KEY = get_key(method="openai")

In [2]:



from langchain.chains import ConversationChain
from langchain import OpenAI

llm = OpenAI(
	temperature=0,
	openai_api_key=OPENAI_API_KEY,
	model_name="text-davinci-003"
)


# now initialize the conversation chain
conversation = ConversationChain(llm=llm)


## lets print a conversation template
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:






We can see the above template. Here **{history}** is what represents the previous conversation.ie, where we use place our Conversational Memory. **{input}** is the current query to the model.

Example:

```
Kerala is a beutiful state in india

human: Hi what is the capital of this place
AI: Trivandrum is the capital of Kerala
```


## Different types of Conversational Memory

### 1. Conversation Buffer Memory

It is the straight forward memory. Here we will pass raw input of past conversation between human and AI as **{history}**

Lets try to use it in a chain

In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory



llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [8]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


' That sounds like an interesting project! What aspects of machine learning are you planning to focus on?'

In [9]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: What is my name?
AI:

> Finished chain.


' Your name is Sam.'

See OpenAI clearly remembers my name. Lets see what it is stored in buffer

In [10]:
print(conversation.memory.buffer)

Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: What is my name?
AI:  Your name is Sam.


You can see that it remembers the entire conversation. This is the conversation buffer memory

**Pros:** 

* No loss of information since evrything is stored
* simple.

**Cons:** 

* More tokens so more cost and slower
* Very long conversations cannot be uses since it can reach token limit.(4096 tokens for text-davinci-003 and gpt-3.5-turbo)


### 2. ConversationBufferWindowMemory

The main difference between ConversationBufferMemory and ConversationBufferWindowMemory is that it adds a window to the buffer.It means that remembers only last k conversations. Lets try with code

Lets use it in a chain

In [17]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory



llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [18]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


' That sounds like an interesting project! What aspects of machine learning are you planning to focus on?'

In [20]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:

> Finished chain.


' That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.'

In [21]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:  That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.
Human: What is my name?
AI:

> Finished chain.


" I'm sorry, I don't know your name."

In [25]:
# from pprint import pprint

# buff = conversation.memory.load_memory_variables(inputs=[])
# pprint(buff)

print(conversation.memory.buffer)

Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:  That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.
Human: What is my name?
AI:  I'm sorry, I don't know your name.


We can see it donot remember the name because it just remebers last 2 conversations.


### 3. ConversationSummary Memory

This type of memory creates a summary of conversation over time. Conversation summary memory summarizes the conversation as it happens and stores the current summary in memory. This memory can then be used to inject the summary of the conversation so far into a prompt/chain. ,this form of memory summarizes the conversation history before it is passed to the {history} parameter

ie after each conversation it creates a summary and this is used as prompt in next conversation

In [2]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory


llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY,model="text-davinci-003",max_tokens=256)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    ## we pass llm here because we need llm to extract entities too
    memory=ConversationSummaryMemory(llm=llm)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [3]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Sam and the AI introduces itself as AI. The AI then asks Sam how they are doing today.
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

' That sounds like an interesting project! Can you tell me more about it?'

In [4]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Sam and the AI introduces itself as AI. The AI then asks Sam how they are doing today, to which Sam responds that they are doing good and are planning to do a college project on machine learning. The AI expresses interest and asks Sam to tell them more about it.
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:

> Finished chain.


" That sounds like a great project! Shap values are a great way to explain the decisions of a machine learning model. They can help to identify which features are most important in a model's predictions, and can also help to identify potential biases in the model."

We can see that from past conversations it created summary and passed as history.

In [6]:
## we can see what is inside the memory

from pprint import pprint
pprint(conversation.memory.buffer)

('\n'
 'The human introduces themselves as Sam and the AI introduces itself as AI. '
 'The AI then asks Sam how they are doing today, to which Sam responds that '
 'they are doing good and are planning to do a college project on machine '
 'learning. The AI expresses interest and asks Sam to tell them more about it, '
 'to which Sam responds that they are planning to focus on model '
 'explainability using shap values. The AI responds that this sounds like a '
 'great project and explains that shap values are a great way to explain the '
 'decisions of a machine learning model, as they can help to identify which '
 "features are most important in a model's predictions and can also help to "
 'identify potential biases in the model.')


**Pros and Cons**

**Pros**

* shortens longer conversations when compared with conversation buffer memory

**Cons**

* Can result in higher token usage for shorter conversations.ie it can create summary of long length for smaller conversations.



### 4. Conversation Summary Buffer Memory


It uses a buffer of recent interactions in memory. In addition instead of flusing out old conversations, it also uses a summary of it.

It uses token length rather than number of interactions to determine when to flush interactions.


ie if we set max_token_limit to a heigher number, we have more recent conversations in out prompt.if we set to low, we will have summary more and very less last conversaion.


In [25]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory


llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY,model="text-davinci-003",max_tokens=256)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    ## we pass llm here because we need llm to extract entities too
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [26]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


' That sounds like an interesting project! What aspects of machine learning are you planning to focus on?'

In [27]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:

> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

' That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.'

In [28]:
conversation.predict(input="Iam also paln to do a seminar on model explainability")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human introduces themselves as Sam and the AI introduces itself as AI. The AI asks how Sam is doing and Sam responds that they are doing good and are planning to do a college project on machine learning.
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:  That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.
Human: Iam also pal

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a


> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

' That sounds like a great idea! Model explainability is an important topic in machine learning, and a seminar would be a great way to share your knowledge with others. What topics do you plan to cover in your seminar?'

In [29]:
conversation.predict(input="Thanks")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human introduces themselves as Sam and the AI introduces itself as AI. The AI asks how Sam is doing and Sam responds that they are doing good and are planning to do a college project on machine learning. The AI expresses interest in the project and Sam reveals they are planning to focus on model explainability using shap values. The AI approves of the project and explains that shap values are a great way to explain the behavior of a machine learning model and can help identify potential areas of improvement.
Human: Iam also paln to do a seminar on model explainability
AI:  That sounds like a great idea! Model explainability is an important topic in machine

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a


> Finished chain.


" You're welcome! Is there anything else I can help you with?"

In athe above conversation also we can see that it tooks summary of past conversations and took the last 2 conversations(based on max_tokens we give) and given as history for LLM.

In [30]:

# from pprint import pprint
# pprint(conversation.memory.buffer)
from pprint import pprint

buff = conversation.memory.load_memory_variables(inputs=[])
pprint(buff)

{'history': 'System: \n'
            'The human introduces themselves as Sam and the AI introduces '
            'itself as AI. The AI asks how Sam is doing and Sam responds that '
            'they are doing good and are planning to do a college project on '
            'machine learning. The AI expresses interest in the project and '
            'Sam reveals they are planning to focus on model explainability '
            'using shap values. The AI approves of the project and explains '
            'that shap values are a great way to explain the behavior of a '
            'machine learning model and can help identify potential areas of '
            'improvement.\n'
            'Human: Iam also paln to do a seminar on model explainability\n'
            'AI:  That sounds like a great idea! Model explainability is an '
            'important topic in machine learning, and a seminar would be a '
            'great way to share your knowledge with others. What topics do you '
        

**Pros and Cons**

**Pros**

* Prevent us loss of information from recent conversation

**Cons**

* More tokens for shorter conversations.
* May have loss of information from past conversations as we are ataking the summary

### 5. Conversation Token Buffer Memory

ConversationTokenBufferMemory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

The major differece with Conversation Buffer Window memory is that it uses number of interactions(window size) where as here it uses number of tokens to define memory.

In [31]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory


llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY,model="text-davinci-003",max_tokens=256)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    ## we pass llm here because we need llm to extract entities too
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [32]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


' That sounds like an interesting project! What aspects of machine learning are you planning to focus on?'

In [33]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi goodmorning, My name is Sam.
AI:  Hi Sam, my name is AI. It's nice to meet you. How are you doing today?
Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:

> Finished chain.


' That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.'

In [34]:
conversation.predict(input="Iam also paln to do a seminar on model explainability")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI:  That sounds like an interesting project! What aspects of machine learning are you planning to focus on?
Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:  That sounds like a great project! Shap values are a great way to explain the behavior of a machine learning model. It can help you understand why the model is making certain predictions and can help you identify potential areas of improvement.
Human: Iam also paln to do a seminar on model explainability
AI:

> Finished chain.


' That sounds like a great idea! Model explainability is an important topic in machine learning, and a seminar would be a great way to share your knowledge and help others understand the concept.'

We can see that it remembers some of its past conversations. Its based on token length we give.

**Pros and Cons**

**Pros**

* can remeber past interactions based on token length

**Cons**

* If token length specified is small,it donot remeber old conversations. So can lead to loss of information.

### 6.Entity Memory

Entity memory remembers given facts about specific entities in a conversation. It extracts information on entities (using an LLM) and builds up its knowledge about that entity over time.

This will automatically save the entities from conversation and use it as history in conversation.Lets see with a conversation chain.

In [35]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE


llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY,model="text-davinci-003",max_tokens=256)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    ## we pass llm here because we need llm to extract entities too
    memory=ConversationEntityMemory(llm=llm)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Hi Sam, nice to meet you! How can I help you today?'

In [36]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a


> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

' That sounds like an interesting project! What would you like to know about machine learning?'

In [29]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a


> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

"\nYes, focusing on model explainability using SHAP values is a great idea! SHAP values are a powerful tool for understanding the impact of each feature on a model's predictions, and can be used to identify areas of improvement and gain insights into how a model works. Additionally, SHAP values can be used to explain the predictions of a model to stakeholders, making them a valuable tool for communicating the results of a machine learning project."

Here along with past conversations extracted entities is also passed.
We can see that from each conversation entities like name, collegename etc and its relevent details are captured and passed as context during the conversation. That is the use of entity memory.



### 7. ConversationKnowledge Graph

This is based on concpt of knowledge graphs.

Key concept: the conversation knowledge graph memory keeps a knowledge graph of all the entities that have been mentioned in the interactions together with their semantic relationships

In [37]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationKGMemory


llm = OpenAI(temperature=0,openai_api_key=OPENAI_API_KEY,model="text-davinci-003",max_tokens=256)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    ## we pass llm here because we need llm to extract entities too
    memory=ConversationKGMemory(llm=llm)
)

conversation.predict(input="hi goodmorning, My name is Sam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi goodmorning, My name is Sam.
AI:

> Finished chain.


" Hi Sam, my name is AI. It's nice to meet you. How are you doing today?"

In [38]:
conversation.predict(input="Iam doing good.Iam planning to do College Project on Machine Learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Iam doing good.Iam planning to do College Project on Machine Learning
AI:

> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

' That sounds like an interesting project! Can you tell me more about it?'

In [39]:
conversation.predict(input="Iam planning to focus on model explainability using shap values.Is it good?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Iam planning to focus on model explainability using shap values.Is it good?
AI:


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a


> Finished chain.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-O9k3eDOJJRBDGSg4pTC8p2vy on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
